In [ ]:
import os
import pandas as pd

def load_data(data_dir='../data/raw', columns_to_load=None, date_parse_columns=['date']):
    """
    Load all CSV files from the specified directory and returns a list of dataframes
    ---
    Returns:
        dataframes: a list of dataframes; each dataframe is a patient's data loaded from the csv files
    """
    csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(os.path.join(data_dir, file), usecols=columns_to_load, parse_dates=date_parse_columns)
        # df['patient_id'] = file.split('_')[0]  # Extract patient ID from filename
        dataframes.append(df)
    
    # combined_df = pd.concat(dataframes, ignore_index=True, axis=0)
    # combined_df = dataframes[0]
    print(f"Loaded data from {len(csv_files)} files.")
    return dataframes

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def preprocess_data(df):
    """
    Preprocess the combined DataFrame.
    ---
    1. Fill NaN values with 0
    2. Replace 0 with 'NULL' in the 'msg_type' column
    3. One hot encode the 'msg_type' column
    4. Drop irrelevant columns for the target variable 
    (in this case, only keeps 'ANNOUNCE_MEAL', 'INTERVENTION_SNACK', 'ANNOUNCE_EXERCISE', 'DOSE_INSULIN', 'DOSE_BASAL_INSULIN')
    5. Drop rows with invalid dates
    6. Change affects_fob and affects_iob to 1 and 0
    """
    df = df.fillna(0)
    df['msg_type'] = df['msg_type'].replace(0, 'NULL')

    # Convert 'date' column to datetime with a custom format
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce', utc=True)
    
    # Drop rows where the date is null or invalid
    df = df.dropna(subset=['date'])
    
    # Sort by date and handle duplicate timestamps
    df = df.sort_values('date')
    
    # Set 'date' as the index
    df.set_index('date', inplace=True)
    # Change affects_fob and affects_iob to 1 and 0
    df['affects_fob'] = df['affects_fob'].apply(lambda x: 1 if x != 0 else 0)
    df['affects_iob'] = df['affects_iob'].apply(lambda x: 1 if x != 0 else 0)

    RELEVANT_MSG_TYPES = ['ANNOUNCE_MEAL', 'INTERVENTION_SNACK', 'ANNOUNCE_EXERCISE', 'DOSE_INSULIN', 'DOSE_BASAL_INSULIN']
    
    encoder = OneHotEncoder(categories='auto', sparse_output=False)
    encoded_data = encoder.fit_transform(df[['msg_type']])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['msg_type']), index=df.index)
    
    df = df.drop(columns=['msg_type'])
    df = pd.concat([df, encoded_df], axis=1)
    
    COLUMNS_TO_DROP = [col for col in df.columns if 'msg_type' in col and not any(msg_type in col for msg_type in RELEVANT_MSG_TYPES)]
    df.drop(columns=COLUMNS_TO_DROP, inplace=True)
    
    return df


In [ ]:
import numpy as np

def create_meal_labels(df):
    """
    Generates labels for each meal segment based on meal announcements and endings.

    This function processes a DataFrame containing meal-related information and creates
    a label array where each meal segment is assigned a unique integer identifier.

    -----------
    Params:
        df : pandas.DataFrame
            The input DataFrame containing meal data. It must have a 'msg_type' column
            with 'ANNOUNCE_MEAL' and 'MEAL_END' entries.

    --------
    Returns:
    
        An array of integers where each element corresponds to a row in the input DataFrame.
        0 indicates no active meal, while positive integers represent unique meal segments.

    Notes:
    ------
    - If multiple meals are active simultaneously, the most recent meal's label is used.
    - Meal segment labels are assigned sequentially starting from 1.

    """
    labels = np.zeros(len(df))
    meal_start = df['msg_type'] == 'ANNOUNCE_MEAL'
    meal_end = df['msg_type'] == 'MEAL_END'
    
    meal_counter = 1
    active_meals = []
    
    for i in range(len(df)):
        if meal_start.iloc[i]:
            active_meals.append(meal_counter)
            meal_counter += 1
        
        if len(active_meals) > 0:
            labels[i] = active_meals[-1]
        
        if meal_end.iloc[i] and len(active_meals) > 0:
            active_meals.pop()
    
    return labels